In [1]:
!pip install fsspec
!pip install gcsfs
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.5 MB/s 
     |████████████████████████████████| 144 kB 47.5 MB/s 
     |████████████████████████████████| 271 kB 54.2 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.9 MB/s 
     |████████████████████████████████| 101 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 37.1 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/s

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/Colab Notebooks/tfm"
import sys
sys.path.append('tfm/checkpoints/')
sys.path.append('tfm/outputs/')

Mounted at /content/drive


In [3]:
from datasets import Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    TFBertForSequenceClassification,
)

In [4]:
filepath = "../../data/outputs/dataset_2021_paragraph"
filepath = "tfm/outputs/dataset_2021_paragraph" # Filepath COLAB
filepath = "https://storage.googleapis.com/tfm_aideas_datasets/dataset_2020_paragraph" # Filepath GCloud

In [5]:
df_total_labels = pd.read_csv(f"{filepath}.csv", sep=";", index_col=0, parse_dates=["date"], low_memory=False)

In [6]:
dataset_opt = Dataset.from_pandas(df_total_labels.loc[:,["body_no_entities","vola_label"]])

A continuación se reentrena únicamente la última capa del modelo BETO para esta tarea de clasificación.

In [10]:
N_LABELS = 2

In [11]:
checkpoint = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/310 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [12]:
model = TFBertForSequenceClassification.from_pretrained(checkpoint, num_labels=N_LABELS)

Downloading:   0%|          | 0.00/512M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model.layers[0].trainable=False

In [14]:
model.compile(
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
optimizer=Adam(learning_rate=0.0001),
metrics=['accuracy']
)

In [15]:
# filepath_checkpoint = './checkpoints/finanzas_tag_s_n/checkpoint'
filepath_checkpoint = 'tfm/checkpoints/finanzas_tag_s_n/checkpoint' # Filepath COLAB

In [16]:
model.load_weights(filepath_checkpoint)
# model.evaluate(x_test.data, y_test)

In [17]:
def encode(input):
    return tokenizer(input["body_no_entities"], padding="max_length", truncation=True)

In [18]:
tokenized_dataset = dataset_opt.map(encode, batched=True)
tf_tokenized_dataset = tokenized_dataset.remove_columns(["pk", "body_no_entities",  "vola_label"])
tf_tokenized_dataset.set_format(type='tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask'])

Parameter 'function'=<function encode at 0x7f62aa1d0ef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/636 [00:00<?, ?ba/s]

In [ ]:
results = model.predict(tf_tokenized_dataset.to_tf_dataset(batch_size=256), verbose=1)

In [ ]:
df_total_labels['finanzas_tag_s_n'] = results.logits.argmax(axis=1)
df_total_labels['finanzas_tag_s_n_confianza'] = tf.nn.softmax(results.logits).numpy().max(axis=1)
df_total_labels['finanzas_tag_s_proba'] = tf.nn.softmax(results.logits).numpy()[:,0]

In [ ]:
filepath = "tfm/outputs/dataset_2020_paragraph"
df_total_labels.to_csv(f'{filepath}_predict_finanzas_tag_s_n.csv', sep=";")